In [4]:
# Dependencies
!pip3 install --upgrade pip
!pip3 install matplotlib
!pip3 install numpy
!pip3 install librosa
!pip3 install pandas

!pip3 install boto3
!pip3 install numba==0.45 # for librosa, https://github.com/librosa/librosa/issues/1160

     |████████████████████████████████| 1.5 MB 4.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
     |████████████████████████████████| 183 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 5.4 MB/s  eta 0:00:01
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201374 sha256=2d359baae4cacfbd2f97834e31bde2d8001e251378b18276f8ded796b5f51560
  Stored in directory: /home/ec2-user/.cache/pip/wheels/32/2c/ce/86e49d4769aceba728421c24c0d726054bf4ca01175ff42bdd
  Created wheel for audioread: filename=audioread-2.1.8-py3-none-any.whl size=23091 sha256=20c8bee1764ce2b20cb6c26efd12a112eb64c43cd2fa2deb634d822bb1e37fe7
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1d/9a/c7/249a5daf5cb90d9786afaec371cba9dc43f04f916db5d1caff
  Created wheel for resampy: filename=resam

In [5]:
!pip install tf-nightly --upgrade # Need nightly version for TFlite conversion

     |████████████████████████████████| 345.3 MB 6.5 kB/s  eta 0:00:01
     |████████████████████████████████| 8.9 MB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 62.8 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 67.2 MB/s eta 0:00:01
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 1.15.2 requires gast==0.2.2, but you'll have gast 0.3.3 which is i

In [7]:
import tensorflow as tf
print("Tensorflow version: {}".format(tf.__version__))

import librosa, librosa.display
import numpy as np

Tensorflow version: 1.15.2


OSError: sndfile library not found

# Load Dataset: only download from s3 IF ./audioData is empty

In [8]:
# Load Dataset
import boto3
import sagemaker
sagemaker_session = sagemaker.Session()

BUCKET = sagemaker_session.default_bucket()
FOLDER = 'live'
OUTPUT_FOLDER = 'live-model-v4'
LOCAL_TRAIN_DATA = './audioData'

In [9]:
import os
if len(os.listdir(LOCAL_TRAIN_DATA) ) == 0:
    !aws s3 cp s3://$BUCKET/$FOLDER/ ./audioData --recursive
else:
    print(len(os.listdir(LOCAL_TRAIN_DATA) ))

23653


### visualize audio

In [10]:
import IPython.display as ipd
import matplotlib.pyplot as plt

filepath = './audioData/A Test2 32446.0-35746.0 iter0.wav'
plt.figure(figsize=(12,4))
data,sample_rate = librosa.load(filepath, sr=None)
_ = librosa.display.waveplot(data,sr=sample_rate)
ipd.Audio(filepath)

NameError: name 'librosa' is not defined

<Figure size 864x288 with 0 Axes>

# Extract audio_data and label

In [8]:
##  EG filename = 'F#m Test3 34330-41240 iter254.wav' , ie excluding LOCAL_TRAIN_DATA folder

CHORDS = ['Noise', 'A', 'Am', 'Bb', 'Bbm', 'B', 'Bm', 'C', 'Cm', 'C#', 'C#m',
          'D', 'Dm', 'D#', 'D#m','E', 'Em', 'F', 'Fm', 'F#', 'F#m', 'G',
          'Gm', 'G#', 'G#m']

def extract_data(filename): # len 8160
    try:
        filepath = '{}/{}'.format(LOCAL_TRAIN_DATA, filename)
        audio, sr = librosa.load(filepath, sr=None, res_type='kaiser_fast')  
    except Exception as e:
        print("Error encountered while parsing file: {} {}".format(filename, e))
        return None 
    return audio

def data_to_chroma(data):
    hop_length = 512
    sr = 48000
    chroma = librosa.feature.chroma_stft(data, sr=sr, hop_length=hop_length)
    chroma = np.mean(chroma.T,axis=0)
    return chroma

## Extract Label
def get_chord_name(filename):
    return filename.split()[0]

def chord_name_to_onehot(c): # O
    # 'Noise' would return all 1.0
    # "not an identifiable chord" will return all 0.0
    return [1.0 if c == chord else 0.0 for chord in CHORDS]

def onehot_to_chord(onehot): # O
    onehot = list(onehot) # convert to python if it is numpy array
    try:
        i = onehot.index(1.0)
    except:
        print("no valid chord from onehot")
    return CHORDS[i]

## For one file C# - Chroma features using Librosa Library. shape = (12,)

In [9]:
filename = 'C# Test3 3827-11140 iter514.wav'
filepath = LOCAL_TRAIN_DATA + '/' + filename

In [10]:
audio = extract_data(filename)
print(audio)
chroma = data_to_chroma(audio)
chroma

[-0.04885864 -0.08361816 -0.09838867 ...  0.12991333  0.13528442
  0.13726807]


array([0.36102372, 0.38479376, 0.33067036, 0.32812068, 0.43780902,
       0.7073642 , 0.45068264, 0.73381793, 1.        , 0.57231593,
       0.28627527, 0.26780984], dtype=float32)

In [11]:
c = get_chord_name(filename)
c

'C#'

In [12]:
onehot = chord_name_to_onehot(c)
print(onehot) # C# index 9
onehot_to_chord(onehot)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


'C#'

# Extract data, labels for all files

## get ~600 samples for each chord to have evenly distributed data. 
### Original distribution of data is uneven: 
```
# from collections import Counter, OrderedDict
# count = Counter()
# for chord in [get_chord_name(clip) for clip in train_files]:
#     count[chord] += 1
# count
# OrderedDict(sorted(count.items(), key=lambda t: t[1]))

OrderedDict([('.ipynb_checkpoints', 1),
             ('F#', 583),
             ('Bm', 591),
             ('Fm', 593),
             ('Bb', 595),
             ('Bbm', 597),
             ('Gm', 597),
             ('C#', 599),
             ('G#m', 599),
             ('Cm', 599),
             ('F#m', 599),
             ('Dm', 600),
             ('D#m', 600),
             ('Noise', 600),
             ('D#', 600),
             ('G#', 600),
             ('A', 900),
             ('D', 1200),
             ('E', 1200),
             ('Em', 1200),
             ('Am', 1200),
             ('B', 1200),
             ('C#m', 1200),
             ('F', 1800),
             ('G', 2400),
             ('C', 2400)])
```

In [13]:
train_files = os.listdir(LOCAL_TRAIN_DATA)

In [14]:
from collections import Counter, OrderedDict
count = Counter()

even_dist_train_files = []
for clip in train_files:
    chord = get_chord_name(clip)
    if count[chord] < 600:
        count[chord] += 1
        even_dist_train_files.append(clip)
        
OrderedDict(sorted(count.items(), key=lambda t: t[1]))

OrderedDict([('.ipynb_checkpoints', 1),
             ('F#', 583),
             ('Bm', 591),
             ('Fm', 593),
             ('Bb', 595),
             ('Bbm', 597),
             ('Gm', 597),
             ('C#', 599),
             ('G#m', 599),
             ('Cm', 599),
             ('F#m', 599),
             ('D', 600),
             ('Dm', 600),
             ('G', 600),
             ('C', 600),
             ('D#m', 600),
             ('E', 600),
             ('Em', 600),
             ('F', 600),
             ('Am', 600),
             ('Noise', 600),
             ('B', 600),
             ('A', 600),
             ('C#m', 600),
             ('D#', 600),
             ('G#', 600)])

## Extract audio data from all files into "features"

In [15]:
features = []
i = 0

for file in even_dist_train_files:
    if not file.startswith('.'):
        data = extract_data(file)
        chord_class = get_chord_name(file)
        features.append([data, chord_class])
        i += 1
        print(i)
    
print("finished parsing files: ", i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
# # Convert into a Panda dataframe 
import pandas as pd
featuresdf = pd.DataFrame(features, columns=['feature','class'])

In [17]:
# CONVERT DATA AND LABELS
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
class_list = featuresdf['class']
y = class_list.tolist() # list of chord labels

In [18]:
# Encode the classification labels to one hot vectors
yy = [chord_name_to_onehot(label) for label in y]
yy = np.array(yy)

## Shuffle and split dataset

In [19]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [20]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(yy.shape[1]) # num labels

(11961, 8160)
(2991, 8160)
(11961, 25)
(2991, 25)
25


In [24]:
x_train[:10]

array([[-2.0004272e-01, -1.9592285e-01, -1.9146729e-01, ...,
         8.6059570e-03,  1.2420654e-02,  1.4434814e-02],
       [-1.7694092e-01, -1.9152832e-01, -2.0111084e-01, ...,
        -1.2207031e-03, -5.0354004e-03, -9.3383789e-03],
       [-8.4594727e-02, -7.9956055e-02, -7.6263428e-02, ...,
        -1.5304565e-01, -1.4086914e-01, -1.3394165e-01],
       ...,
       [ 3.0517578e-05,  9.1552734e-05,  3.0517578e-05, ...,
        -3.9672852e-04, -9.1552734e-05, -1.5258789e-04],
       [ 7.7056885e-02,  7.1289062e-02,  6.1157227e-02, ...,
        -3.7179565e-01, -3.2748413e-01, -3.0020142e-01],
       [-1.1425781e-01, -1.1654663e-01, -1.2109375e-01, ...,
         5.2185059e-02,  5.2612305e-02,  5.2917480e-02]], dtype=float32)

In [25]:
with open('x_train_small.npy', 'wb') as f:
    np.save(f, x_train[:10])
with open('x_test_small.npy', 'wb') as f:
    np.save(f, x_train[10:15])
    
with open('y_train_small.npy', 'wb') as f:
    np.save(f, x_train[:10])
with open('y_test_small.npy', 'wb') as f:
    np.save(f, x_train[10:15])

In [23]:
with open('x_train_small.npy', 'rb') as f:
    x_train_small = np.load(f)
print(x_train_small)

[[-2.0004272e-01 -1.9592285e-01 -1.9146729e-01 ...  8.6059570e-03
   1.2420654e-02  1.4434814e-02]
 [-1.7694092e-01 -1.9152832e-01 -2.0111084e-01 ... -1.2207031e-03
  -5.0354004e-03 -9.3383789e-03]
 [-8.4594727e-02 -7.9956055e-02 -7.6263428e-02 ... -1.5304565e-01
  -1.4086914e-01 -1.3394165e-01]
 ...
 [ 3.0517578e-05  9.1552734e-05  3.0517578e-05 ... -3.9672852e-04
  -9.1552734e-05 -1.5258789e-04]
 [ 7.7056885e-02  7.1289062e-02  6.1157227e-02 ... -3.7179565e-01
  -3.2748413e-01 -3.0020142e-01]
 [-1.1425781e-01 -1.1654663e-01 -1.2109375e-01 ...  5.2185059e-02
   5.2612305e-02  5.2917480e-02]]


# Custom Keras Layer AudioToChromaLayerLARGE
transforms raw audio into chromas averaged across time. Using CQT

See more indepth exploration of chroma extraction: https://oslyn-le-v1.notebook.ca-central-1.sagemaker.aws/notebooks/oslyn-LE/Live-model-v4/Librosa%20Features%20Analysis.ipynb

Input = (batchsize, 8160), Output = (batchsize, 12)

### Note 1: Used magic numbers for `K`, `nfft` in `def CQT_fast` to avoid np/tf math calculations
assumed sample rate 48,000, sample length 170ms
we can calculate it from scratch, the code is commented on those lines

### Note 2: Used `scipy.fftpack fft` in creating the sparse kernel because it can (to handle the math operations) but have to use `ft = tf.signal.fft(x)` on the input tensor because cannot convert tensor to numpy/

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, PReLU, LayerNormalization
from sklearn import metrics 

Couldnt use librosa due to numpy tensor errors. 
Also couldnt use https://github.com/zafarrafii/Z#cqtkernel-constant-q-transform-cqt-kernel

In [28]:
import math
from scipy.signal import hamming
from scipy.fftpack import fft

class AudioToChromaLayerLARGE(tf.keras.layers.Layer):
    """input is raw audio of size (*, 8160), output chromagram of size (*, 12)."""

    def __init__(self, **kwargs):
        super(AudioToChromaLayerLARGE, self).__init__(**kwargs)
        
    def call(self, audio): # LOGIC
        apply_to_batch = tf.map_fn(self.compute_chroma, audio, fn_output_signature=tf.TensorSpec((12,), dtype=tf.float32))
        return apply_to_batch
    
    def get_config(self):
        config = super(AudioToChromaLayerLARGE, self).get_config()
        return config
    
#     def log2(self, x):
#         x = tf.cast(x, tf.float16)
#         two = tf.constant(2, tf.float16)
#         return tf.math.log(x)/tf.math.log(two)
    
#     def nearestPow2(self, inp):
#         power = tf.math.ceil(self.log2(inp))
#         return 2**power

    """Function to compute CQT using sparse matrix multiplication, Brown and Puckette 1992- fast"""	
    def CQT_fast(self, x,fs,bins,fmin,fmax,M):
        threshold = 0.0054 #for Hamming window
        K = 48          # int(bins*np.ceil(self.log2(fmax/fmin)))
        Q = 1/(2**(1/bins)-1)
        nfft = 8192     # tf.cast(self.nearestPow2(tf.math.ceil(Q*fs/fmin)), dtype=tf.int32)
        tempKernel = np.zeros(nfft, dtype = np.complex)
        specKernel = np.zeros(nfft, dtype = np.complex)
        sparKernel = []

        #create sparse Kernel 
        for k in range(K-1,-1,-1):
            fk = (2**(k/bins))*fmin
            N = np.int32(np.round((Q*fs)/fk))
            tempKernel[:N] = hamming(N)/N * np.exp(-2*np.pi*1j*Q*np.arange(N)/N)
            specKernel = fft(tempKernel)
            specKernel[np.where(np.abs(specKernel) <= threshold)] = 0
            if k == K-1:
                sparKernel = specKernel
            else:
                sparKernel = np.vstack((specKernel, sparKernel))
        sparKernel = np.transpose(np.conjugate(sparKernel))/nfft
        sparKernel = tf.convert_to_tensor(sparKernel, dtype=tf.complex64)
        x = tf.cast(x, tf.complex64)
        padding = tf.constant([[16, 16]]) # 4112 = (nfft - x.shape[0])/2
        x = np.pad(x, padding, mode='constant')
        ft = tf.signal.fft(x)
        cqt = tf.tensordot(ft, sparKernel,axes=1)
        return cqt

    @tf.function(input_signature=[tf.TensorSpec(shape=(8160,), dtype=tf.float32)])
    def compute_chroma(self, x, fs=48000):
        fmin = 110
        fmax = 1760
        bins = 12
        M = 1
        nOctave = np.int32(np.ceil(np.log2(fmax/fmin)))
        CH = np.zeros(bins)
        #Compute constant Q transform
        cqt_fast = self.CQT_fast(x,fs,bins,fmin,fmax,M)
        ## Compute Pitch Class Profile from constant Q transform
        cqt_abs = tf.math.abs(cqt_fast)
        reshaped = tf.reshape(cqt_abs, [-1, bins])
        return tf.reduce_sum(reshaped, 0)


# Test just AudioToChromaLayer, batched

In [29]:
notes = ['A','A#','B','C','C#','D','D#','E','F','F#','G','G#']

x_Bbm, sample_rate = librosa.load('./24chords/F#m.wav', sr=None)
x_C, sample_rate = librosa.load('./24chords/C.wav', sr=None)
x_batch = np.array([x_Bbm, x_C])
layer = AudioToChromaLayerLARGE()
# activation = Activation('softmax')
chroma = layer.call(x_batch)
# chroma = norm.call(x_batch)
print(chroma[0])
print(chroma[1])

plt.xticks(np.arange(12),notes)
plt.title('F#m')
plt.plot(chroma[0]) # Plot list. x-values assumed to be [0, 1, 2, 3]
plt.show()
plt.xticks(np.arange(12),notes)
plt.title('C')
plt.plot(chroma[1]) # Plot list. x-values assumed to be [0, 1, 2, 3]
plt.show()

NotImplementedError: in user code:

    <ipython-input-26-8de17001358b>:67 compute_chroma  *
        cqt_fast = self.CQT_fast(x,fs,bins,fmin,fmax,M)
    <ipython-input-28-18a61cca59da>:53 CQT_fast  *
        x = np.pad(x, padding, mode='constant')
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/lib/arraypad.py:1168 pad  **
        if not np.asarray(pad_width).dtype.kind == 'i':
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/numeric.py:538 asarray
        return array(a, dtype, copy=False, order=order)
    /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:849 __array__
        " a NumPy call, which is not supported".format(self.name))

    NotImplementedError: Cannot convert a symbolic Tensor (Const_1:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported


['jhngbfvdcs giubhj  # Realised: The Y-axis range varies widely, F#m had the highest values resulting in the model only predicting F#m.

### Solution: Need to normalise each sample within the layer
We can use Keras' [LayerNormalization()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LayerNormalization) after AudioToChromaLayer

Normalize the activations of the previous layer for each given example in a batch independently, rather than across a batch like Batch Normalization. i.e. applies a transformation that maintains the mean activation within each example close to 0 and the activation standard deviation close to 1.

# THE MODEL (with LayerNormalisation after AudioToChromaLayer)

In [ ]:
# # # Construct model 
num_labels = yy.shape[1]

model = Sequential()
model.add(AudioToChromaLayerLARGE(input_shape=(8160,), name="chroma"))

model.add(LayerNormalization()) ### NECESSARY!!! Cos Chromas all diff y scale resulting only 1 prediction - the one with the highest

model.add(Dense(64, name="dense1"))
model.add(PReLU())

model.add(Dense(32, name="dense2"))
model.add(PReLU())

model.add(Dense(32, name="dense3"))
model.add(PReLU())

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
# Display model architecture summary 
model.build(input_shape=(8160,))
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)

### Pre-training accuracy, predictions

In [ ]:
# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

# ACTUAL Training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 25
num_batch_size = 64

model_path = "./model/epoch{epoch:02d}-acc{val_accuracy:.2f}.hdf5" # Use this to save each epoch where accuracy improves

checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_accuracy',
                               verbose=1, save_best_only=True, mode='max')
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

# Predict / Validate

In [ ]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

In [ ]:
CHORDS = ['Noise', 'A', 'Am', 'Bb', 'Bbm', 'B', 
          'Bm', 'C', 'Cm', 'C#', 'C#m', 'D',
          'Dm', 'D#', 'D#m','E', 'Em', 'F',
          'Fm', 'F#', 'F#m', 'G', 'Gm', 'G#',
          'G#m']

def predict_extract_features(filename): ## NOTE NP.array at the end, diff from extract_features
    hop_length = 512
    try:
        filepath = '{}/{}'.format(LOCAL_TRAIN_DATA, filename)
        audio, sr = librosa.load(filepath, sr=None, res_type='kaiser_fast') 
    except Exception as e:
        print("Error encountered while parsing file: {} {}".format(filename, e))
        return None 
    return np.array([audio])

def predict(file_name):
    feature = predict_extract_features(file_name)
    predicted_vector = model.predict(feature) 
    predicted_proba = predicted_vector[0]
    predicted_i = np.argmax(predicted_proba)
    return CHORDS[predicted_i]

def predict_proba(file_name):
    feature = predict_extract_features(file_name)
    predicted_vector = model.predict(feature) 
    predicted_proba = predicted_vector[0]
    predicted_i = np.argmax(predicted_proba)

    # to see probabilities of each chord:
    for i in range(len(predicted_proba)):
        category = CHORDS[i]
        print(category, "\t\t : ", format(predicted_proba[i], '.32f'))
    
    return CHORDS[predicted_i]

In [ ]:
filename = 'A Test2 32446.0-35746.0 iter0.wav'
print('predicted: ' + predict_proba(filename) + '\n')

filename = 'C# Test3 3827-11140 iter514.wav'
print('predicted: ' + predict_proba(filename) + '\n')

filename = 'F# Test3 34330-41240 iter254.wav'
print('predicted: ' + predict_proba(filename) + '\n')

In [ ]:
val_files = even_dist_train_files[:200]

val_files.sort()

print('chord   prediction')
for i in range(len(val_files)):
    file = val_files[i]
    actual = get_chord_name(file)
    pred = predict(file)
    print(i, actual, ' '*(6-len(actual)), pred)

# Convert keras .hdf5 file to .tflite

### Problem 1: AudioToChromaLayerLARGE not found
Solution: add `custom_objects` in `tf.keras.models.load_model`

###  Problem 2: Not all tf operations are supported in tf.lite
Solution:
```
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
```

### Problem 3: `map_fn`:
Solution: https://github.com/tensorflow/tensorflow/issues/40221
Use TF 2.3.0 (experimental) and `@tf.function(input_signature=[tf.TensorSpec(shape=(8160,), dtype=tf.float32)])`

In [13]:
model_best = tf.keras.models.load_model('./model/epoch24-acc0.99.hdf5',
                        custom_objects={'AudioToChromaLayerLARGE': AudioToChromaLayerLARGE})

In [20]:
converter1 = tf.lite.TFLiteConverter.from_keras_model(model_best)
converter1.experimental_new_converter = True
converter1.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

tflite_model1 = converter1.convert()

file = open('model_builtins.tflite' , 'wb' ) 
file.write(tflite_model1)

INFO:tensorflow:Assets written to: /tmp/tmpc2hyev0l/assets


INFO:tensorflow:Assets written to: /tmp/tmpc2hyev0l/assets


ConverterError: /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py:1089:0: error: 'tf.FusedBatchNormV3' op is neither a custom op nor a flex op
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py:1183:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/lite/python/lite.py:810:0: note: called from
<ipython-input-20-3a990714fb94>:5:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IP

In [19]:
converter2 = tf.lite.TFLiteConverter.from_keras_model(model_best)
converter2.experimental_new_converter = True
converter2.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model2 = converter2.convert()

file = open('model_both.tflite' , 'wb' ) 
file.write(tflite_model2)

INFO:tensorflow:Assets written to: /tmp/tmpxia9jzyn/assets


INFO:tensorflow:Assets written to: /tmp/tmpxia9jzyn/assets


3179200

In [21]:
converter3 = tf.lite.TFLiteConverter.from_keras_model(model_best)
converter3.experimental_new_converter = True
converter3.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model3 = converter3.convert()

file = open('model_select.tflite' , 'wb' ) 
file.write(tflite_model2)

INFO:tensorflow:Assets written to: /tmp/tmpllqut9oj/assets


INFO:tensorflow:Assets written to: /tmp/tmpllqut9oj/assets


ConverterError: /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507:0: error: 'tfl.while' op is a TFLite builtin op but builtin emission is not enabled
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:574:0: note: called from
<ipython-input-7-cf0e73ae6b83>:12:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:617:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:985:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:504:0: note: called from
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:382:0: note: called from
/home/ec2-user/anaconda3/en

### [Archive]  Alternative model.save

In [ ]:
# need model.save ....
model.save('./model2')
model = tf.keras.models.load_model('./model2', custom_objects={'AudioToChromaLayerLARGE': AudioToChromaLayerLARGE})
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# file = open('model.tflite' , 'wb' ) 
# file.write(tflite_model)



# [Archive] Failed Models (using Librosa.feature.chroma_stft)
Works for one layer, ie I pass in the input (8160,) and get the correct chromagram output of (12,)
But throws building, saving errors when put into the whole Model.

In [ ]:
## TRY 1:
class AudioToChromagram(tf.keras.layers.Layer):
    """input is raw audio of size (*, 8160), output chromagram of size (*, 12)."""

    def __init__(self): # SAVE CONFIG IN MEMBER_VARIABLES
        super(AudioToChromagram, self).__init__(trainable=False)
        
    def chroma_stft(self, clip):
        return librosa.feature.chroma_stft(clip, sr=48000, hop_length=512)

    def call(self, audio): # LOGIC
        # print('layer input ', audio.shape)
        print('is model eager? ', tf.executing_eagerly())
        audio_numpy = audio.numpy()
        # print('audio_numpy ', audio_numpy.shape)
        if audio_numpy.ndim == 1:
            chroma = self.chroma_stft(audio_numpy)
            chroma_ave = np.mean(chroma.T,axis=0)
        elif audio_numpy.ndim == 2: # call stft on each sample
            chroma = np.apply_along_axis(self.chroma_stft, 1, audio_numpy)
            chroma_ave = np.mean(chroma, axis=2)
        else:
            raise Exception('audio_numpy should be (*, 8160) or (8160,)')
        return tf.convert_to_tensor(chroma_ave)

In [ ]:
## COMPARE WITH LIBROSA LIBRARY - works well!! Checked inputs and outputs to this layer

filepath1 = './audioData/A Test2 32446.0-35746.0 iter0.wav'
audio1, sr1 = librosa.load(filepath1, sr=None)

filepath2 = './audioData/C# Test3 3827-11140 iter514.wav'
audio2, sr2 = librosa.load(filepath2, sr=None)

# BATCHED samples
sample_data = np.array([audio1, audio2])
layer = AudioToChromagram()
output = layer(sample_data)
print(output)

## SINGLE sample
layer = AudioToChromagram()
output = layer(audio1)
print(output)

When we run just the layer, `tf.executing_eagerly() == True` . But when we put it in the whole model, `tf.executing_eagerly() == False` so we run into the Numpy-Tensor conversion issues and the model can't build.

Have tried using all sorts of ways to fix this: TF 2 (eager by default), 
```import tensorflow as tf; tf.enable_eager_execution()``` or 
```model.compile(run_eagerly=True)```
but seems to be a known bug in 2020. That a Layer can run eagerly but is not eager when put into a Sequential Model
https://github.com/tensorflow/tensorflow/issues/38038
https://github.com/tensorflow/tensorflow/issues/38775

In [ ]:
# inputs = Input(shape=(8160,))
# chroma = AudioToChromagram()
# x = chroma(inputs)
# dense1 = Dense(64, activation='relu')
# x = dense1(x)
# dense2 = Dense(64, activation="relu")
# x = dense2(x)
# outputs = Dense(num_labels)(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs, name="audio_model")

TRY 2: Different way of writing the same model - can build, train and "save"
But turns out it's only building/saving weights without the model, so the actual model can't save and be converted to tflite

In [ ]:
import tensorflow as tf
num_labels = 25

## TRY 2

class ChordModel(tf.keras.Model):
    
    def chroma_stft(self, clip):
        return librosa.feature.chroma_stft(clip, sr=48000, hop_length=512)
    
    def audio_to_chroma(self, audio):
        # print(tf.executing_eagerly())
        audio_numpy = audio.numpy()
        if audio_numpy.ndim == 1:
            chroma = self.chroma_stft(audio_numpy)
            chroma_ave = np.mean(chroma.T,axis=0)
        elif audio_numpy.ndim == 2: # call stft on each sample
            chroma = np.apply_along_axis(self.chroma_stft, 1, audio_numpy)
            chroma_ave = np.mean(chroma, axis=2)
        else:
            raise Exception('audio_numpy should be (*, 8160) or (8160,)')
        return tf.convert_to_tensor(chroma_ave)
    
    def __init__(self, inputs, **kwargs):
        super(ChordModel, self).__init__(inputs, **kwargs)
        self.chromagram = Lambda(self.audio_to_chroma)
        self.dense1 = Dense(64, activation='relu')
        self.drop1 = Dropout(0.25)
        self.dense2 = Dense(128, activation='relu')
        self.drop2 = Dropout(0.25)
        self.dense3 = Dense(64, activation='relu')
        self.dense4 = Dense(num_labels, activation='softmax')
        self.activate = Activation('softmax')
        
    def call(self, inputs):
#         print('model: inputs shape ', inputs.shape)
        x = self.chromagram(inputs)
        x = self.dense1(x)
        x = self.drop1(x)
        x = self.dense2(x)
        x = self.drop2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.activate(x)

inp = Input(shape=(8160,))
model = ChordModel(inputs=inp, dynamic=True)

# [Reference]

In [ ]:
CHORDS = [
          'Noise', 'A', 'Am', 'Bb', 'Bbm', 'B', 
          'Bm', 'C', 'Cm', 'C#', 'C#m', 'D',
          'Dm', 'D#', 'D#m','E', 'Em', 'F',
          'Fm', 'F#', 'F#m', 'G', 'Gm', 'G#',
          'G#m']

chord_templates = {'A':   [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 'Am':  [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 'A#':  [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 'A#m': [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 'B':   [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 'Bm':  [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 'C':   [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
 'Cm':  [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
 'C#':  [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
 'C#m': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
 'D':   [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 'Dm':  [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 'D#':  [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 'D#m': [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 'E':   [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
 'Em':  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 'F':   [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 'Fm':  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
 'F#':  [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 'F#m': [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 'G':   [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
 'Gm':  [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
 'G#':  [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
 'G#m': [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]}